## LLM의 답변을 원하는 형태로 조정하는 Output Parser

실무에서 LLM을 API로 활용하면서 구조화된 답변을 받고 싶을때 대개의 LLM은 문장을 생성하는 것에 특화되어 있어 정해진 형식으로 답변을 받는것은 어렵다.  
이런 경우, 프롬프트 엔지니어링이 일반적인 해결책이다.  
영화 추천 AI앱을 만든다고 했을 때, AI가 추천해주는 영화 제목들은 일반 텍스트가 아니라 리스트 형태로 보여줘야 한다.   
이를 가정하고 ChatPromptTemplate 기반 프롬프트 엔지니어링 후 LLM API로 리스트 형태의 답변을 받아보자.

In [2]:
import os 
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.prompts import HumanMessagePromptTemplate 
from langchain_core.messages import SystemMessage 
from langchain_core.prompts import ChatPromptTemplate

api_key=os.environ["GEMINI_API_KEY"]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", temperature=0, google_api_key=api_key)

# ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
         HumanMessagePromptTemplate.from_template("{text}")             
    ]
)

messages = chat_template.format_messages(text="스릴러 영화 3개를 추천해줘.")
answer = llm.invoke(messages)
result = answer.content 
print(result)

["세븐", "양들의 침묵", "기생충"]


SystemMessage에서 정의한대로 리스트 형태로 출력했다.  
JSON이나 Pydantic 같은 더 복잡한 구조의 답변을 받기 위해서는 Output Parser를 활용해서 정해진 형식의 답변을 출력할 수 있다.  
 - CSV 파서, Datetime 파서, JSON 파서, Pydantic 파서  

In [4]:
# 쉼표로 구분된 리스트를 출력하는 CSV 파서

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI 

api_key=os.environ["GEMINI_API_KEY"]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", temperature=0, google_api_key=api_key)

# csv파서 선언
output_parser = CommaSeparatedListOutputParser()
# csv 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿의 partial_variables에 csv 형식 지정 프롬프트 주입
prompt = PromptTemplate(
    template="List {number} {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject", "number"],
    partial_variables={"format_instructions": format_instructions},
)

# 프롬프트 탬플릿-모델-Output Parser를 체인으로 연결
chain = prompt | model | output_parser
chain.invoke({"subject":"SF영화", "number":"3"})

['스타워즈', '매트릭스', '인터스텔라']

In [5]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [7]:
# 날짜 형식만 출력하는 datetime 파서 

from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI 

output_parser = DatetimeOutputParser()

template = """ 
    Answer the user's question: {question}
    
    {format_instructions}
"""

prompt = PromptTemplate.from_template(
    template, 
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", temperature=0, google_api_key=api_key)

chain = prompt | model | output_parser 
output = chain.invoke({"question": "chatgpt는 언제 개발됐어?"})
print(output)

2024-01-15 14:15:16.000001


In [8]:
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1829-05-26T00:05:16.509109Z, 1079-06-15T09:41:33.917568Z, 1970-04-07T20:30:42.571030Z\n\nReturn ONLY this string, no other words!"

In [13]:
# 시스템 통신의 기본 형식을 위한 JSON 파서 

from typing import List 
from langchain.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI 

# 원하는 데이터 구조 정의 
class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: int = Field(description="사용자가 물어본 나라의 인구 수(int 형식)")
# and a query intented to prompt a language model to populate the data structure 
country_query = "차드는 어떤 나라야?"

# Set up a parser + inject instructions into the prompt template 
parser = JsonOutputParser(pydantic_object=Country) 

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", temperature=0, google_api_key=api_key)

chain = prompt | model | parser 

chain.invoke({'query': country_query})

{'continent': '아프리카', 'population': 17000000}